In [1]:
(require booleans redex/reduction-semantics racket/pretty 
        (for-syntax syntax/parse racket/base))
(define-syntax define/ppl 
 (syntax-parser 
   [(_ name body) 
    #'(begin (define-term name body)
             (for-each pretty-write (term name)))]))


The COS circuit for:
```
(signal S1
    (present S1
     (seq (emit S2)
          (present S2
             nothing
             (emit S1)))
     nothing))
```

In [2]:
(define/ppl a-circuit
   ([START = true]
    [present-S1-else = (and (not S1) START)]
    [present-S1-then = (and S1 START)]
    [S2 = present-S1-then]
    [present-S2-then = (and S2 present-S1-then)]
    [present-S2-else = (and (not S2) present-S1-then)]
    [S1 = present-S2-else]
    [present-S2-term = (or present-S2-else present-S2-else)]
    [term = (or present-S1-else
                present-S2-term)]))

(START = true)
(present-S1-else = (and (not S1) START))
(present-S1-then = (and S1 START))
(S2 = present-S1-then)
(present-S2-then = (and S2 present-S1-then))
(present-S2-else = (and (not S2) present-S1-then))
(S1 = present-S2-else)
(present-S2-term = (or present-S2-else present-S2-else))
(term = (or present-S1-else present-S2-term))


In [3]:
(define/ppl b-circuit (convert-P a-circuit))

((+ START) = true)
((- START) = false)
((+ present-S1-else) = (and (- S1) (+ START)))
((- present-S1-else) = (or (+ S1) (- START)))
((+ present-S1-then) = (and (+ S1) (+ START)))
((- present-S1-then) = (or (- S1) (- START)))
((+ S2) = (+ present-S1-then))
((- S2) = (- present-S1-then))
((+ present-S2-then) = (and (+ S2) (+ present-S1-then)))
((- present-S2-then) = (or (- S2) (- present-S1-then)))
((+ present-S2-else) = (and (- S2) (+ present-S1-then)))
((- present-S2-else) = (or (+ S2) (- present-S1-then)))
((+ S1) = (+ present-S2-else))
((- S1) = (- present-S2-else))
((+ present-S2-term) = (or (+ present-S2-else) (+ present-S2-else)))
((- present-S2-term) = (and (- present-S2-else) (- present-S2-else)))
((+ term) = (or (+ present-S1-else) (+ present-S2-term)))
((- term) = (and (- present-S1-else) (- present-S2-term)))


Classical logic on b-circuit

In [4]:
(define/ppl step2-classic
  (propigate/remove* b-circuit (+ START) (- START)))

((+ present-S1-else) = (and (- S1) true))
((- present-S1-else) = (or (+ S1) false))
((+ present-S1-then) = (and (+ S1) true))
((- present-S1-then) = (or (- S1) false))
((+ S2) = (+ present-S1-then))
((- S2) = (- present-S1-then))
((+ present-S2-then) = (and (+ S2) (+ present-S1-then)))
((- present-S2-then) = (or (- S2) (- present-S1-then)))
((+ present-S2-else) = (and (- S2) (+ present-S1-then)))
((- present-S2-else) = (or (+ S2) (- present-S1-then)))
((+ S1) = (+ present-S2-else))
((- S1) = (- present-S2-else))
((+ present-S2-term) = (or (+ present-S2-else) (+ present-S2-else)))
((- present-S2-term) = (and (- present-S2-else) (- present-S2-else)))
((+ term) = (or (+ present-S1-else) (+ present-S2-term)))
((- term) = (and (- present-S1-else) (- present-S2-term)))


In [5]:
(define/ppl step3-classic
  (propigate/remove* step2-classic
                     (- present-S1-then) (+ present-S1-then)
                     (- present-S1-else) (+ present-S1-else)))

((+ S2) = (and (+ S1) true))
((- S2) = (or (- S1) false))
((+ present-S2-then) = (and (+ S2) (and (+ S1) true)))
((- present-S2-then) = (or (- S2) (or (- S1) false)))
((+ present-S2-else) = (and (- S2) (and (+ S1) true)))
((- present-S2-else) = (or (+ S2) (or (- S1) false)))
((+ S1) = (+ present-S2-else))
((- S1) = (- present-S2-else))
((+ present-S2-term) = (or (+ present-S2-else) (+ present-S2-else)))
((- present-S2-term) = (and (- present-S2-else) (- present-S2-else)))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or (+ S1) false) (- present-S2-term)))


In [6]:
(define/ppl step4-classic
  (propigate/remove* step3-classic
                     (- present-S2-then) (+ present-S2-then)
                     (- present-S2-else) (+ present-S2-else)))

((+ S2) = (and (+ S1) true))
((- S2) = (or (- S1) false))
((+ S1) = (and (- S2) (and (+ S1) true)))
((- S1) = (or (+ S2) (or (- S1) false)))
((+ present-S2-term)
 =
 (or (and (- S2) (and (+ S1) true)) (and (- S2) (and (+ S1) true))))
((- present-S2-term)
 =
 (and (or (+ S2) (or (- S1) false)) (or (+ S2) (or (- S1) false))))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or (+ S1) false) (- present-S2-term)))


In [7]:
(define/ppl step5-classic
  (replace* step4-classic
            [(and (+ S1) true) (+ S1)]
            [(or (- S1) false) (- S1)]))

((+ S2) = (+ S1))
((- S2) = (- S1))
((+ S1) = (and (- S2) (+ S1)))
((- S1) = (or (+ S2) (- S1)))
((+ present-S2-term) = (or (and (- S2) (+ S1)) (and (- S2) (+ S1))))
((- present-S2-term) = (and (or (+ S2) (- S1)) (or (+ S2) (- S1))))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or (+ S1) false) (- present-S2-term)))


In [8]:
(define/ppl step6-classic
 (propigate/remove* step5-classic 
                    (- S2) 
                    (+ S2)))

((+ S1) = (and (- S1) (+ S1)))
((- S1) = (or (+ S1) (- S1)))
((+ present-S2-term) = (or (and (- S1) (+ S1)) (and (- S1) (+ S1))))
((- present-S2-term) = (and (or (+ S1) (- S1)) (or (+ S1) (- S1))))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or (+ S1) false) (- present-S2-term)))


In [9]:
(define/ppl step7-classic 
 (replace* step6-classic
           [(and (- S1) (+ S1)) false]))

((+ S1) = false)
((- S1) = (or (+ S1) (- S1)))
((+ present-S2-term) = (or false false))
((- present-S2-term) = (and (or (+ S1) (- S1)) (or (+ S1) (- S1))))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or (+ S1) false) (- present-S2-term)))


In [10]:
(define/ppl step8-classic (propigate* step7-classic (+ S1)))

((+ S1) = false)
((- S1) = (or false (- S1)))
((+ present-S2-term) = (or false false))
((- present-S2-term) = (and (or false (- S1)) (or false (- S1))))
((+ term) = (or (and (- S1) true) (+ present-S2-term)))
((- term) = (and (or false false) (- present-S2-term)))


In [11]:
(define/ppl step9-classic 
 (replace* step8-classic
           [(or false (- S1)) (- S1)]
           [(and (- S1) true) (- S1)]
           [(or false false) false]))

((+ S1) = false)
((- S1) = (- S1))
((+ present-S2-term) = false)
((- present-S2-term) = (and (- S1) (- S1)))
((+ term) = (or (- S1) (+ present-S2-term)))
((- term) = (and false (- present-S2-term)))


In [12]:
(define/ppl step10-classic
  (propigate/remove* step9-classic (+ present-S2-term)))

((+ S1) = false)
((- S1) = (- S1))
((- present-S2-term) = (and (- S1) (- S1)))
((+ term) = (or (- S1) false))
((- term) = (and false (- present-S2-term)))


In [13]:
(define/ppl step11-classic
  (replace* step10-classic
            [(or (- S1) false) (- S1)]
            [(and false (- present-S2-term)) (- present-S2-term)]
            [(and (- S1) (- S1)) (- S1)]))

((+ S1) = false)
((- S1) = (- S1))
((- present-S2-term) = (- S1))
((+ term) = (- S1))
((- term) = (- present-S2-term))


In [14]:
(define/ppl step12-classic
  (propigate/remove* step11-classic
                     (- present-S2-term)))

((+ S1) = false)
((- S1) = (- S1))
((+ term) = (- S1))
((- term) = (- S1))


This last step is purely classical: we guess that `(- S1)` could be false. (It can't be true, because that would make `+term` and `-term` true)

In [15]:
(define/ppl step13-classic
  (replace* step12-classic
            [(- S1) false]))

((+ S1) = false)
((- S1) = false)
((+ term) = false)
((- term) = false)


Now we start constructive logic solving on `a-circuit`

In [16]:
(define/ppl step2 (propigate/remove* a-circuit START))

(present-S1-else = (and (not S1) true))
(present-S1-then = (and S1 true))
(S2 = present-S1-then)
(present-S2-then = (and S2 present-S1-then))
(present-S2-else = (and (not S2) present-S1-then))
(S1 = present-S2-else)
(present-S2-term = (or present-S2-else present-S2-else))
(term = (or present-S1-else present-S2-term))


In [17]:
(define/ppl step3 
 (propigate/remove* step2 
                    present-S1-then
                    present-S1-else))

(S2 = (and S1 true))
(present-S2-then = (and S2 (and S1 true)))
(present-S2-else = (and (not S2) (and S1 true)))
(S1 = present-S2-else)
(present-S2-term = (or present-S2-else present-S2-else))
(term = (or (and (not S1) true) present-S2-term))


In [18]:
(define/ppl step4 
  (propigate/remove* step3 
                     present-S2-then
                     present-S2-else))

(S2 = (and S1 true))
(S1 = (and (not S2) (and S1 true)))
(present-S2-term
 =
 (or (and (not S2) (and S1 true)) (and (not S2) (and S1 true))))
(term = (or (and (not S1) true) present-S2-term))


In [19]:
(define/ppl step5 
 (replace* step4 
           [(and S1 true) S1]))

(S2 = S1)
(S1 = (and (not S2) S1))
(present-S2-term = (or (and (not S2) S1) (and (not S2) S1)))
(term = (or (and (not S1) true) present-S2-term))


In [20]:
(define/ppl step6 
 (propigate/remove* step5 S2))

(S1 = (and (not S1) S1))
(present-S2-term = (or (and (not S1) S1) (and (not S1) S1)))
(term = (or (and (not S1) true) present-S2-term))


In [23]:
(define/ppl step7
 (replace* step6 
           [(and (not S1) S1) S1];; this could also be false, but that breaks?
           [(and (not S1) true) (not S1)]))

(S1 = S1)
(present-S2-term = (or S1 S1))
(term = (or (not S1) present-S2-term))


In [24]:
(define/ppl step8
  (propigate/remove* step7 present-S2-term))

(S1 = S1)
(term = (or (not S1) (or S1 S1)))


In [25]:
(define/ppl step9
  (replace* step8 
            [(or S1 S1) S1]))

(S1 = S1)
(term = (or (not S1) S1))


In [26]:
(define/ppl step9-classic (convert-P step9))

((+ S1) = (+ S1))
((- S1) = (- S1))
((+ term) = (or (- S1) (+ S1)))
((- term) = (and (+ S1) (- S1)))
